# Structured Outputs For Reasoning Models

When working with reasoning models that use special tokens like `<think>...</think>` to denote reasoning sections, you might want to allow free-form text within these sections while still enforcing grammar constraints on the rest of the output.

SGLang provides a feature to disable grammar restrictions within reasoning sections. This is particularly useful for models that need to perform complex reasoning steps before providing a structured output.

To enable this feature, use the `--reasoning-parser` flag which decide the think_end_token, such as `</think>`, when launching the server. You can also specify the reasoning parser using the `--reasoning-parser` flag.

## Supported Models

Currently, SGLang supports the following reasoning models:
- [DeepSeek R1 series](https://huggingface.co/collections/deepseek-ai/deepseek-r1-678e1e131c0169c0bc89728d): The reasoning content is wrapped with `<think>` and `</think>` tags.
- [QwQ](https://huggingface.co/Qwen/QwQ-32B): The reasoning content is wrapped with `<think>` and `</think>` tags.


## Usage

## OpenAI Compatible API

Specify the `--grammar-backend`, `--reasoning-parser` option.

In [1]:
import openai
import os
from sglang.test.test_utils import is_in_ci

if is_in_ci():
    from patch import launch_server_cmd
else:
    from sglang.utils import launch_server_cmd

from sglang.utils import wait_for_server, print_highlight, terminate_process

os.environ["TOKENIZERS_PARALLELISM"] = "false"


server_process, port = launch_server_cmd(
    "python -m sglang.launch_server --model-path deepseek-ai/DeepSeek-R1-Distill-Qwen-7B --host 0.0.0.0 --reasoning-parser deepseek-r1"
)

wait_for_server(f"http://localhost:{port}")
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

[2025-05-04 20:24:36] server_args=ServerArgs(model_path='deepseek-ai/DeepSeek-R1-Distill-Qwen-7B', tokenizer_path='deepseek-ai/DeepSeek-R1-Distill-Qwen-7B', tokenizer_mode='auto', skip_tokenizer_init=False, enable_tokenizer_batch_encode=False, load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='deepseek-ai/DeepSeek-R1-Distill-Qwen-7B', chat_template=None, completion_template=None, is_embedding=False, revision=None, host='0.0.0.0', port=37727, mem_fraction_static=0.88, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=889524572, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_

[2025-05-04 20:24:45] Attention backend not set. Use fa3 backend by default.
[2025-05-04 20:24:45] Init torch distributed begin.


[2025-05-04 20:24:45] Init torch distributed ends. mem usage=0.00 GB
[2025-05-04 20:24:45] Load weight begin. avail mem=53.73 GB


[2025-05-04 20:24:46] Ignore import error when loading sglang.srt.models.llama4. 


[2025-05-04 20:24:46] Using model weights format ['*.safetensors']
Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


...

[2025-05-04 20:24:50] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=131072


[2025-05-04 20:24:50] INFO:     Started server process [4046910]
[2025-05-04 20:24:50] INFO:     Waiting for application startup.
[2025-05-04 20:24:50] INFO:     Application startup complete.
[2025-05-04 20:24:50] INFO:     Uvicorn running on http://0.0.0.0:37727 (Press CTRL+C to quit)


[2025-05-04 20:24:51] INFO:     127.0.0.1:56136 - "GET /v1/models HTTP/1.1" 200 OK


[2025-05-04 20:24:51] INFO:     127.0.0.1:56142 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-05-04 20:24:51] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-04 20:24:54] INFO:     127.0.0.1:56148 - "POST /generate HTTP/1.1" 200 OK
[2025-05-04 20:24:54] The server is fired up and ready to roll!


### JSON

you can directly define a JSON schema or use [Pydantic](https://docs.pydantic.dev/latest/) to define and validate the response.

**Using Pydantic**

In [2]:
from pydantic import BaseModel, Field


# Define the schema using Pydantic
class CapitalInfo(BaseModel):
    name: str = Field(..., pattern=r"^\w+$", description="Name of the capital city")
    population: int = Field(..., description="Population of the capital city")


response = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    messages=[
        {
            "role": "user",
            "content": "Please generate the information of the capital of France in the JSON format.",
        },
    ],
    temperature=0,
    max_tokens=2048,
    response_format={
        "type": "json_schema",
        "json_schema": {
            "name": "foo",
            # convert the pydantic model to json schema
            "schema": CapitalInfo.model_json_schema(),
        },
    },
)

print_highlight(
    f"reasoing_content: {response.choices[0].message.reasoning_content}\n\ncontent: {response.choices[0].message.content}"
)

[2025-05-04 20:24:57] Prefill batch. #new-seq: 1, #new-token: 18, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-04 20:24:58] Decode batch. #running-req: 1, #token: 52, token usage: 0.00, gen throughput (token/s): 4.85, #queue-req: 0


[2025-05-04 20:24:58] Decode batch. #running-req: 1, #token: 92, token usage: 0.00, gen throughput (token/s): 107.69, #queue-req: 0


[2025-05-04 20:24:59] Decode batch. #running-req: 1, #token: 132, token usage: 0.01, gen throughput (token/s): 107.12, #queue-req: 0


[2025-05-04 20:24:59] Decode batch. #running-req: 1, #token: 172, token usage: 0.01, gen throughput (token/s): 107.40, #queue-req: 0


...

[2025-05-04 20:25:17] Decode batch. #running-req: 1, #token: 1932, token usage: 0.09, gen throughput (token/s): 64.94, #queue-req: 0


[2025-05-04 20:25:17] Decode batch. #running-req: 1, #token: 1972, token usage: 0.10, gen throughput (token/s): 63.66, #queue-req: 0


[2025-05-04 20:25:18] Decode batch. #running-req: 1, #token: 2012, token usage: 0.10, gen throughput (token/s): 59.35, #queue-req: 0


[2025-05-04 20:25:19] Decode batch. #running-req: 1, #token: 2052, token usage: 0.10, gen throughput (token/s): 55.56, #queue-req: 0


[2025-05-04 20:25:19] INFO:     127.0.0.1:56152 - "POST /v1/chat/completions HTTP/1.1" 200 OK


**JSON Schema Directly**


In [3]:
import json

json_schema = json.dumps(
    {
        "type": "object",
        "properties": {
            "name": {"type": "string", "pattern": "^[\\w]+$"},
            "population": {"type": "integer"},
        },
        "required": ["name", "population"],
    }
)

response = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    messages=[
        {
            "role": "user",
            "content": "Give me the information of the capital of France in the JSON format.",
        },
    ],
    temperature=0,
    max_tokens=2048,
    response_format={
        "type": "json_schema",
        "json_schema": {"name": "foo", "schema": json.loads(json_schema)},
    },
)

print_highlight(
    f"reasoing_content: {response.choices[0].message.reasoning_content}\n\ncontent: {response.choices[0].message.content}"
)

[2025-05-04 20:25:19] Prefill batch. #new-seq: 1, #new-token: 17, #cached-token: 2, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-04 20:25:20] Decode batch. #running-req: 1, #token: 44, token usage: 0.00, gen throughput (token/s): 45.47, #queue-req: 0


[2025-05-04 20:25:20] Decode batch. #running-req: 1, #token: 84, token usage: 0.00, gen throughput (token/s): 65.04, #queue-req: 0


[2025-05-04 20:25:21] Decode batch. #running-req: 1, #token: 124, token usage: 0.01, gen throughput (token/s): 65.47, #queue-req: 0


[2025-05-04 20:25:22] Decode batch. #running-req: 1, #token: 164, token usage: 0.01, gen throughput (token/s): 65.59, #queue-req: 0


...

[2025-05-04 20:25:39] Decode batch. #running-req: 1, #token: 1924, token usage: 0.09, gen throughput (token/s): 106.39, #queue-req: 0


[2025-05-04 20:25:40] Decode batch. #running-req: 1, #token: 1964, token usage: 0.10, gen throughput (token/s): 105.65, #queue-req: 0


[2025-05-04 20:25:40] Decode batch. #running-req: 1, #token: 2004, token usage: 0.10, gen throughput (token/s): 105.87, #queue-req: 0


[2025-05-04 20:25:40] Decode batch. #running-req: 1, #token: 2044, token usage: 0.10, gen throughput (token/s): 105.47, #queue-req: 0


[2025-05-04 20:25:41] INFO:     127.0.0.1:56152 - "POST /v1/chat/completions HTTP/1.1" 200 OK


### EBNF

In [4]:
ebnf_grammar = """
root ::= city | description
city ::= "London" | "Paris" | "Berlin" | "Rome"
description ::= city " is " status
status ::= "the capital of " country
country ::= "England" | "France" | "Germany" | "Italy"
"""

response = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    messages=[
        {"role": "system", "content": "You are a helpful geography bot."},
        {
            "role": "user",
            "content": "Give me the information of the capital of France.",
        },
    ],
    temperature=0,
    max_tokens=2048,
    extra_body={"ebnf": ebnf_grammar},
)

print_highlight(
    f"reasoing_content: {response.choices[0].message.reasoning_content}\n\ncontent: {response.choices[0].message.content}"
)

[2025-05-04 20:25:41] Prefill batch. #new-seq: 1, #new-token: 21, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-04 20:25:41] Decode batch. #running-req: 1, #token: 39, token usage: 0.00, gen throughput (token/s): 78.85, #queue-req: 0


[2025-05-04 20:25:41] Decode batch. #running-req: 1, #token: 79, token usage: 0.00, gen throughput (token/s): 109.64, #queue-req: 0


[2025-05-04 20:25:42] Decode batch. #running-req: 1, #token: 119, token usage: 0.01, gen throughput (token/s): 108.72, #queue-req: 0
[2025-05-04 20:25:42] INFO:     127.0.0.1:56152 - "POST /v1/chat/completions HTTP/1.1" 200 OK


### Regular expression

In [5]:
response = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    messages=[
        {"role": "user", "content": "What is the capital of France?"},
    ],
    temperature=0,
    max_tokens=2048,
    extra_body={"regex": "(Paris|London)"},
)

print_highlight(
    f"reasoing_content: {response.choices[0].message.reasoning_content}\n\ncontent: {response.choices[0].message.content}"
)

[2025-05-04 20:25:42] Prefill batch. #new-seq: 1, #new-token: 10, #cached-token: 2, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-04 20:25:42] Decode batch. #running-req: 1, #token: 33, token usage: 0.00, gen throughput (token/s): 98.74, #queue-req: 0


[2025-05-04 20:25:42] Decode batch. #running-req: 1, #token: 73, token usage: 0.00, gen throughput (token/s): 107.69, #queue-req: 0


[2025-05-04 20:25:43] Decode batch. #running-req: 1, #token: 113, token usage: 0.01, gen throughput (token/s): 108.50, #queue-req: 0


[2025-05-04 20:25:43] Decode batch. #running-req: 1, #token: 153, token usage: 0.01, gen throughput (token/s): 109.35, #queue-req: 0


[2025-05-04 20:25:43] Decode batch. #running-req: 1, #token: 193, token usage: 0.01, gen throughput (token/s): 109.84, #queue-req: 0


[2025-05-04 20:25:44] Decode batch. #running-req: 1, #token: 233, token usage: 0.01, gen throughput (token/s): 109.54, #queue-req: 0


[2025-05-04 20:25:44] Decode batch. #running-req: 1, #token: 273, token usage: 0.01, gen throughput (token/s): 109.12, #queue-req: 0


[2025-05-04 20:25:45] Decode batch. #running-req: 1, #token: 313, token usage: 0.02, gen throughput (token/s): 109.37, #queue-req: 0


[2025-05-04 20:25:45] Decode batch. #running-req: 1, #token: 353, token usage: 0.02, gen throughput (token/s): 108.86, #queue-req: 0
[2025-05-04 20:25:45] INFO:     127.0.0.1:56152 - "POST /v1/chat/completions HTTP/1.1" 200 OK


### Structural Tag

In [6]:
tool_get_current_weather = {
    "type": "function",
    "function": {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "city": {
                    "type": "string",
                    "description": "The city to find the weather for, e.g. 'San Francisco'",
                },
                "state": {
                    "type": "string",
                    "description": "the two-letter abbreviation for the state that the city is"
                    " in, e.g. 'CA' which would mean 'California'",
                },
                "unit": {
                    "type": "string",
                    "description": "The unit to fetch the temperature in",
                    "enum": ["celsius", "fahrenheit"],
                },
            },
            "required": ["city", "state", "unit"],
        },
    },
}

tool_get_current_date = {
    "type": "function",
    "function": {
        "name": "get_current_date",
        "description": "Get the current date and time for a given timezone",
        "parameters": {
            "type": "object",
            "properties": {
                "timezone": {
                    "type": "string",
                    "description": "The timezone to fetch the current date and time for, e.g. 'America/New_York'",
                }
            },
            "required": ["timezone"],
        },
    },
}

schema_get_current_weather = tool_get_current_weather["function"]["parameters"]
schema_get_current_date = tool_get_current_date["function"]["parameters"]


def get_messages():
    return [
        {
            "role": "system",
            "content": f"""
# Tool Instructions
- Always execute python code in messages that you share.
- When looking for real time information use relevant functions if available else fallback to brave_search
You have access to the following functions:
Use the function 'get_current_weather' to: Get the current weather in a given location
{tool_get_current_weather["function"]}
Use the function 'get_current_date' to: Get the current date and time for a given timezone
{tool_get_current_date["function"]}
If a you choose to call a function ONLY reply in the following format:
<{{start_tag}}={{function_name}}>{{parameters}}{{end_tag}}
where
start_tag => `<function`
parameters => a JSON dict with the function argument name as key and function argument value as value.
end_tag => `</function>`
Here is an example,
<function=example_function_name>{{"example_name": "example_value"}}</function>
Reminder:
- Function calls MUST follow the specified format
- Required parameters MUST be specified
- Only call one function at a time
- Put the entire function call reply on one line
- Always add your sources when using search results to answer the user query
You are a helpful assistant.""",
        },
        {
            "role": "user",
            "content": "You are in New York. Please get the current date and time, and the weather.",
        },
    ]


messages = get_messages()

response = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    messages=messages,
    response_format={
        "type": "structural_tag",
        "max_new_tokens": 2048,
        "structures": [
            {
                "begin": "<function=get_current_weather>",
                "schema": schema_get_current_weather,
                "end": "</function>",
            },
            {
                "begin": "<function=get_current_date>",
                "schema": schema_get_current_date,
                "end": "</function>",
            },
        ],
        "triggers": ["<function="],
    },
)

print_highlight(
    f"reasoing_content: {response.choices[0].message.reasoning_content}\n\ncontent: {response.choices[0].message.content}"
)

[2025-05-04 20:25:45] Prefill batch. #new-seq: 1, #new-token: 471, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-04 20:25:46] Decode batch. #running-req: 1, #token: 495, token usage: 0.02, gen throughput (token/s): 55.03, #queue-req: 0


[2025-05-04 20:25:46] Decode batch. #running-req: 1, #token: 535, token usage: 0.03, gen throughput (token/s): 105.68, #queue-req: 0


[2025-05-04 20:25:46] Decode batch. #running-req: 1, #token: 575, token usage: 0.03, gen throughput (token/s): 104.94, #queue-req: 0


[2025-05-04 20:25:47] Decode batch. #running-req: 1, #token: 615, token usage: 0.03, gen throughput (token/s): 105.20, #queue-req: 0


[2025-05-04 20:25:47] Decode batch. #running-req: 1, #token: 655, token usage: 0.03, gen throughput (token/s): 101.23, #queue-req: 0


[2025-05-04 20:25:48] Decode batch. #running-req: 1, #token: 695, token usage: 0.03, gen throughput (token/s): 98.71, #queue-req: 0


[2025-05-04 20:25:48] Decode batch. #running-req: 1, #token: 735, token usage: 0.04, gen throughput (token/s): 94.31, #queue-req: 0


[2025-05-04 20:25:48] Decode batch. #running-req: 1, #token: 1, token usage: 0.00, gen throughput (token/s): 94.18, #queue-req: 0
[2025-05-04 20:25:48] INFO:     127.0.0.1:56152 - "POST /v1/chat/completions HTTP/1.1" 200 OK


## Native API and SGLang Runtime (SRT)

### JSON

**Using Pydantic**

In [7]:
import requests
from pydantic import BaseModel, Field
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-7B")


# Define the schema using Pydantic
class CapitalInfo(BaseModel):
    name: str = Field(..., pattern=r"^\w+$", description="Name of the capital city")
    population: int = Field(..., description="Population of the capital city")


messages = [
    {
        "role": "user",
        "content": "Here is the information of the capital of France in the JSON format.\n",
    }
]
text = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
# Make API request
response = requests.post(
    f"http://localhost:{port}/generate",
    json={
        "text": text,
        "sampling_params": {
            "temperature": 0,
            "max_new_tokens": 2048,
            "json_schema": json.dumps(CapitalInfo.model_json_schema()),
        },
    },
)
print(response.json())


reasoing_content = response.json()["text"].split("</think>")[0]
content = response.json()["text"].split("</think>")[1]
print_highlight(f"reasoing_content: {reasoing_content}\n\ncontent: {content}")

[2025-05-04 20:25:49] Prefill batch. #new-seq: 1, #new-token: 19, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-04 20:25:50] Decode batch. #running-req: 1, #token: 60, token usage: 0.00, gen throughput (token/s): 32.45, #queue-req: 0


[2025-05-04 20:25:50] Decode batch. #running-req: 1, #token: 100, token usage: 0.00, gen throughput (token/s): 107.16, #queue-req: 0


[2025-05-04 20:25:50] Decode batch. #running-req: 1, #token: 140, token usage: 0.01, gen throughput (token/s): 105.86, #queue-req: 0


[2025-05-04 20:25:51] Decode batch. #running-req: 1, #token: 180, token usage: 0.01, gen throughput (token/s): 102.55, #queue-req: 0


...

[2025-05-04 20:26:08] Decode batch. #running-req: 1, #token: 1900, token usage: 0.09, gen throughput (token/s): 104.54, #queue-req: 0


[2025-05-04 20:26:08] Decode batch. #running-req: 1, #token: 1940, token usage: 0.09, gen throughput (token/s): 105.59, #queue-req: 0


[2025-05-04 20:26:09] Decode batch. #running-req: 1, #token: 1980, token usage: 0.10, gen throughput (token/s): 104.64, #queue-req: 0


[2025-05-04 20:26:09] Decode batch. #running-req: 1, #token: 2020, token usage: 0.10, gen throughput (token/s): 105.76, #queue-req: 0


[2025-05-04 20:26:09] Decode batch. #running-req: 1, #token: 2060, token usage: 0.10, gen throughput (token/s): 101.20, #queue-req: 0
[2025-05-04 20:26:09] INFO:     127.0.0.1:58076 - "POST /generate HTTP/1.1" 200 OK
{'text': 'Okay, so I need to provide the information about the capital of France in JSON format. Hmm, I\'m not entirely sure about all the details, but I\'ll try to think it through.\n\nFirst, I know that the capital of France is Paris. That\'s pretty much a given, right? But I should double-check that. Maybe I can recall any other capitals I know. London is the capital of the UK, Rome is Italy, and maybe Tokyo is Japan\'s. Yeah, Paris seems correct for France.\n\nNow, moving on to the population. I think Paris is a very large city, but I\'m not sure of the exact number. I remember it\'s over 3 million, but I\'m not certain. Maybe around 3.5 million? I should probably look that up, but since I can\'t right now, I\'ll go with 3,500,000 as an estimate.\n\nNext, the area. Par

**JSON Schema Directly**

In [8]:
json_schema = json.dumps(
    {
        "type": "object",
        "properties": {
            "name": {"type": "string", "pattern": "^[\\w]+$"},
            "population": {"type": "integer"},
        },
        "required": ["name", "population"],
    }
)

# JSON
text = tokenizer.apply_chat_template(text, tokenize=False, add_generation_prompt=True)
response = requests.post(
    f"http://localhost:{port}/generate",
    json={
        "text": text,
        "sampling_params": {
            "temperature": 0,
            "max_new_tokens": 2048,
            "json_schema": json_schema,
        },
    },
)

print_highlight(response.json())

[2025-05-04 20:26:09] Prefill batch. #new-seq: 1, #new-token: 3, #cached-token: 2, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-04 20:26:10] Decode batch. #running-req: 1, #token: 37, token usage: 0.00, gen throughput (token/s): 98.26, #queue-req: 0


[2025-05-04 20:26:10] Decode batch. #running-req: 1, #token: 77, token usage: 0.00, gen throughput (token/s): 106.18, #queue-req: 0


[2025-05-04 20:26:11] Decode batch. #running-req: 1, #token: 117, token usage: 0.01, gen throughput (token/s): 91.22, #queue-req: 0


[2025-05-04 20:26:11] Decode batch. #running-req: 1, #token: 157, token usage: 0.01, gen throughput (token/s): 105.27, #queue-req: 0


...

[2025-05-04 20:26:15] Decode batch. #running-req: 1, #token: 637, token usage: 0.03, gen throughput (token/s): 105.51, #queue-req: 0


[2025-05-04 20:26:16] Decode batch. #running-req: 1, #token: 677, token usage: 0.03, gen throughput (token/s): 102.76, #queue-req: 0


[2025-05-04 20:26:16] Decode batch. #running-req: 1, #token: 717, token usage: 0.04, gen throughput (token/s): 104.51, #queue-req: 0


[2025-05-04 20:26:17] Decode batch. #running-req: 1, #token: 757, token usage: 0.04, gen throughput (token/s): 104.60, #queue-req: 0


[2025-05-04 20:26:17] INFO:     127.0.0.1:55084 - "POST /generate HTTP/1.1" 200 OK


### EBNF

In [9]:
response = requests.post(
    f"http://localhost:{port}/generate",
    json={
        "text": "Give me the information of the capital of France.",
        "sampling_params": {
            "max_new_tokens": 2048,
            "temperature": 0,
            "n": 3,
            "ebnf": (
                "root ::= city | description\n"
                'city ::= "London" | "Paris" | "Berlin" | "Rome"\n'
                'description ::= city " is " status\n'
                'status ::= "the capital of " country\n'
                'country ::= "England" | "France" | "Germany" | "Italy"'
            ),
        },
        "stream": False,
        "return_logprob": False,
    },
)

print(response.json())

[2025-05-04 20:26:17] Prefill batch. #new-seq: 1, #new-token: 10, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-05-04 20:26:17] Decode batch. #running-req: 1, #token: 0, token usage: 0.00, gen throughput (token/s): 99.87, #queue-req: 0
[2025-05-04 20:26:17] Prefill batch. #new-seq: 3, #new-token: 3, #cached-token: 30, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-04 20:26:18] Decode batch. #running-req: 3, #token: 134, token usage: 0.01, gen throughput (token/s): 203.16, #queue-req: 0


[2025-05-04 20:26:18] Decode batch. #running-req: 3, #token: 254, token usage: 0.01, gen throughput (token/s): 299.92, #queue-req: 0


[2025-05-04 20:26:18] Decode batch. #running-req: 3, #token: 374, token usage: 0.02, gen throughput (token/s): 301.75, #queue-req: 0


[2025-05-04 20:26:19] Decode batch. #running-req: 3, #token: 494, token usage: 0.02, gen throughput (token/s): 300.64, #queue-req: 0


[2025-05-04 20:26:19] Decode batch. #running-req: 3, #token: 3, token usage: 0.00, gen throughput (token/s): 297.65, #queue-req: 0
[2025-05-04 20:26:19] INFO:     127.0.0.1:55092 - "POST /generate HTTP/1.1" 200 OK
[{'text': "\nThe capital of France is Paris.\n\nThat's all the information I have.\n\nOkay, so I need to figure out the capital of France. I know that Paris is the capital, but I'm not entirely sure. Let me think about why I think that. I've heard it mentioned a lot, especially in movies and TV shows. People often go there for business or tourism. Also, I remember learning in school that Paris is a major city in France, known for landmarks like the Eiffel Tower and the Louvre Museum. Those places are famous worldwide, which makes me think that Paris is indeed the capital. Maybe I can cross-check this with some other sources or my notes. Wait, I don't have any other information right now, but based on what I know, Paris is the capital of France. I don't recall any other major 

### Regular expression

In [10]:
response = requests.post(
    f"http://localhost:{port}/generate",
    json={
        "text": "Paris is the capital of",
        "sampling_params": {
            "temperature": 0,
            "max_new_tokens": 2048,
            "regex": "(France|England)",
        },
    },
)
print(response.json())

[2025-05-04 20:26:19] Prefill batch. #new-seq: 1, #new-token: 5, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-04 20:26:20] Decode batch. #running-req: 1, #token: 46, token usage: 0.00, gen throughput (token/s): 100.74, #queue-req: 0


[2025-05-04 20:26:20] Decode batch. #running-req: 1, #token: 86, token usage: 0.00, gen throughput (token/s): 106.56, #queue-req: 0


[2025-05-04 20:26:20] Decode batch. #running-req: 1, #token: 126, token usage: 0.01, gen throughput (token/s): 106.51, #queue-req: 0


[2025-05-04 20:26:21] Decode batch. #running-req: 1, #token: 166, token usage: 0.01, gen throughput (token/s): 106.21, #queue-req: 0


...

[2025-05-04 20:26:37] Decode batch. #running-req: 1, #token: 1886, token usage: 0.09, gen throughput (token/s): 105.46, #queue-req: 0


[2025-05-04 20:26:38] Decode batch. #running-req: 1, #token: 1926, token usage: 0.09, gen throughput (token/s): 106.13, #queue-req: 0


[2025-05-04 20:26:38] Decode batch. #running-req: 1, #token: 1966, token usage: 0.10, gen throughput (token/s): 105.44, #queue-req: 0


[2025-05-04 20:26:38] Decode batch. #running-req: 1, #token: 2006, token usage: 0.10, gen throughput (token/s): 105.69, #queue-req: 0


[2025-05-04 20:26:39] Decode batch. #running-req: 1, #token: 2046, token usage: 0.10, gen throughput (token/s): 105.89, #queue-req: 0
[2025-05-04 20:26:39] INFO:     127.0.0.1:56638 - "POST /generate HTTP/1.1" 200 OK
{'text': ' France, and the \n\\( n \\)  \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\(

### Structural Tag

In [11]:
text = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
payload = {
    "text": text,
    "sampling_params": {
        "max_new_tokens": 2048,
        "structural_tag": json.dumps(
            {
                "type": "structural_tag",
                "structures": [
                    {
                        "begin": "<function=get_current_weather>",
                        "schema": schema_get_current_weather,
                        "end": "</function>",
                    },
                    {
                        "begin": "<function=get_current_date>",
                        "schema": schema_get_current_date,
                        "end": "</function>",
                    },
                ],
                "triggers": ["<function="],
            }
        ),
    },
}


# Send POST request to the API endpoint
response = requests.post(f"http://localhost:{port}/generate", json=payload)
print_highlight(response.json())

[2025-05-04 20:26:39] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 19, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-04 20:26:39] Decode batch. #running-req: 1, #token: 52, token usage: 0.00, gen throughput (token/s): 100.85, #queue-req: 0


[2025-05-04 20:26:39] Decode batch. #running-req: 1, #token: 92, token usage: 0.00, gen throughput (token/s): 107.95, #queue-req: 0


[2025-05-04 20:26:40] Decode batch. #running-req: 1, #token: 132, token usage: 0.01, gen throughput (token/s): 107.85, #queue-req: 0


[2025-05-04 20:26:40] Decode batch. #running-req: 1, #token: 172, token usage: 0.01, gen throughput (token/s): 106.75, #queue-req: 0


...

[2025-05-04 20:26:48] Decode batch. #running-req: 1, #token: 1012, token usage: 0.05, gen throughput (token/s): 101.86, #queue-req: 0


[2025-05-04 20:26:49] Decode batch. #running-req: 1, #token: 1052, token usage: 0.05, gen throughput (token/s): 104.80, #queue-req: 0


[2025-05-04 20:26:49] Decode batch. #running-req: 1, #token: 1092, token usage: 0.05, gen throughput (token/s): 103.88, #queue-req: 0


[2025-05-04 20:26:49] Decode batch. #running-req: 1, #token: 1132, token usage: 0.06, gen throughput (token/s): 104.67, #queue-req: 0


[2025-05-04 20:26:50] INFO:     127.0.0.1:32812 - "POST /generate HTTP/1.1" 200 OK
[2025-05-04 20:26:50] Decode batch. #running-req: 1, #token: 0, token usage: 0.00, gen throughput (token/s): 106.83, #queue-req: 0


In [12]:
terminate_process(server_process)

[2025-05-04 20:26:50] Child process unexpectedly failed with an exit code 9. pid=4047377
[2025-05-04 20:26:50] Child process unexpectedly failed with an exit code 9. pid=4047301


## Offline Engine API

In [13]:
import sglang as sgl

llm = sgl.Engine(
    model_path="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    reasoning_parser="deepseek-r1",
    grammar_backend="xgrammar",
)

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


Loading safetensors checkpoint shards:  50% Completed | 1/2 [00:01<00:01,  1.42s/it]


Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:02<00:00,  1.35s/it]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:02<00:00,  1.36s/it]



### JSON

**Using Pydantic**

In [14]:
import json
from pydantic import BaseModel, Field


prompts = [
    "Give me the information of the capital of China in the JSON format.",
    "Give me the information of the capital of France in the JSON format.",
    "Give me the information of the capital of Ireland in the JSON format.",
]


# Define the schema using Pydantic
class CapitalInfo(BaseModel):
    name: str = Field(..., pattern=r"^\w+$", description="Name of the capital city")
    population: int = Field(..., description="Population of the capital city")


sampling_params = {
    "temperature": 0,
    "top_p": 0.95,
    "max_new_tokens": 2048,
    "json_schema": json.dumps(CapitalInfo.model_json_schema()),
}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Give me the information of the capital of China in the JSON format.
Generated text: 
Sure, here's the information about the capital of China, Beijing, in JSON format:

```json
{
  "name": "Beijing",
  "capital": "Yes",
  "population": "Over 30 million",
  "founded": "1248",
  "Nickname": "The Heaven on Earth",
  "Location": "Northern China",
  "OfficialLanguages": [
    "Mandarin Chinese",
    "Bingyuan Chinese",
    "Tibetan",
    "Hui",
    "Mongolian",
    "Yugoslav",
    "Other"
  ],
  "KeySights": [
    "The Great Wall",
    "Tiananmen Square",
    "Forbidden City",
    "Beijing Museum",
    "Yuanmingyuan"
  ],
  "Climate": "Temperate"
}
```

Let me know if you need any other information!
Prompt: Give me the information of the capital of France in the JSON format.
Generated text: 
Sure! Here's the information about the capital of France, Paris, in JSON format:

```json
{
  "name": "Paris",
  "country": "France",
  "coordinates": {
    "latitude": 48.8566,
    "longitude": 

**JSON Schema Directly**

In [15]:
prompts = [
    "Give me the information of the capital of China in the JSON format.",
    "Give me the information of the capital of France in the JSON format.",
    "Give me the information of the capital of Ireland in the JSON format.",
]

json_schema = json.dumps(
    {
        "type": "object",
        "properties": {
            "name": {"type": "string", "pattern": "^[\\w]+$"},
            "population": {"type": "integer"},
        },
        "required": ["name", "population"],
    }
)

sampling_params = {"temperature": 0, "max_new_tokens": 2048, "json_schema": json_schema}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Give me the information of the capital of China in the JSON format.
Generated text: 
Sure! Here's the information about the capital of China, Beijing, in JSON format:

```json
{
  "name": "Beijing",
  "capital": "Yes",
  "population": "Over 30 million",
  "founded": "1248",
  "Nickname": "The Heaven on Earth",
  "Location": "Northern China",
  "OfficialLanguages": [
    "Mandarin Chinese",
    "Bingyuan Chinese",
    "Tibetan",
    "Hui",
    "Mongolian",
    "Yugoslav",
    "Other"
  ],
  "KeySights": [
    "The Great Wall",
    "Forbidden City",
    "Tiananmen Square",
    "Beijing Museum",
    "Yuanmingyuan"
  ],
  "Climate": "Temperate"
}
```

Let me know if you need any other information!
Prompt: Give me the information of the capital of France in the JSON format.
Generated text: 
Sure! Here's the information about the capital of France, Paris, in JSON format:

```json
{
  "name": "Paris",
  "country": "France",
  "coordinates": {
    "latitude": 48.8566,
    "longitude": 

### EBNF


In [16]:
prompts = [
    "Give me the information of the capital of France.",
    "Give me the information of the capital of Germany.",
    "Give me the information of the capital of Italy.",
]

sampling_params = {
    "temperature": 0.8,
    "top_p": 0.95,
    "ebnf": (
        "root ::= city | description\n"
        'city ::= "London" | "Paris" | "Berlin" | "Rome"\n'
        'description ::= city " is " status\n'
        'status ::= "the capital of " country\n'
        'country ::= "England" | "France" | "Germany" | "Italy"'
    ),
}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Give me the information of the capital of France.
Generated text: 
The capital of France is Paris.

That's it.

Alright, so I need to figure out how to determine the capital of France. I know that Paris is the capital, but I'm not entirely sure why or how that's determined. I remember that capitals are usually the largest cities in a country, but I think there's more to it. Maybe it's based on historical significance or political importance. I also recall that some countries have capitals that aren't their largest cities, like Germany's Berlin or Brazil's Brasília. So, why is Paris considered the capital?

Perhaps it's because Paris has a rich history and has been
Prompt: Give me the information of the capital of Germany.
Generated text: 
Alright, I need to figure out the information about the capital of Germany. Okay, let's start. I know the capital is Berlin, but I'm not entirely sure about all the details. Maybe I should think about what I've heard before.

First, I remember

### Regular expression

In [17]:
prompts = [
    "Please provide information about London as a major global city:",
    "Please provide information about Paris as a major global city:",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95, "regex": "(France|England)"}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Please provide information about London as a major global city:
Generated text:  its location, major landmarks, cultural significance, economic strength, and natural beauty.92 words.

Sure, I need to provide a 92-word information about London as a major global city. I should cover location, major landmarks, cultural significance, economic strength, and natural beauty.

First, location: London is located in England, United Kingdom, straddling the River Thames.

Major landmarks: The Tower of London, Big Ben, London Eye, British Museum, and the London Underground are significant.

Cultural significance: London is known for its rich history, being the capital of the UK and a cultural melting pot with diverse festivals
Prompt: Please provide information about Paris as a major global city:
Generated text:  include population, language, famous landmarks, cultural significance, and transportation options.

1. Population: Paris has a population of over 2 million people, but it's actuall

In [18]:
text = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
prompts = [text]


sampling_params = {
    "temperature": 0.8,
    "top_p": 0.95,
    "max_new_tokens": 2048,
    "structural_tag": json.dumps(
        {
            "type": "structural_tag",
            "structures": [
                {
                    "begin": "<function=get_current_weather>",
                    "schema": schema_get_current_weather,
                    "end": "</function>",
                },
                {
                    "begin": "<function=get_current_date>",
                    "schema": schema_get_current_date,
                    "end": "</function>",
                },
            ],
            "triggers": ["<function="],
        }
    ),
}


# Send POST request to the API endpoint
outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: <｜begin▁of▁sentence｜><｜User｜>Here is the information of the capital of France in the JSON format.
<｜Assistant｜><think>

Generated text: Alright, so the user is asking for the information about the capital of France in JSON format. They might be a developer working on a project that requires structured data or maybe a student working on a geography assignment. Either way, they need a clear and concise way to present the information.

First, I should figure out what exactly they need. They want the JSON structure, so I need to make sure all the relevant details are included. Let me think about the key pieces of information. The city name is straightforward, it's Paris. Coordinates are important for mapping purposes, so I'll include the latitude and longitude.

I should also consider other fields like population, which can vary, but the approximate number is around 2 million. Adding the country it's located in makes sense for context. Maybe they want to include the timezone, which

In [19]:
llm.shutdown()